# Algoritmo de maximización de la esperanza para el modelo de mezclas Gaussianas

<img style="float: right; margin: 0px 0px 15px 15px;" src="https://upload.wikimedia.org/wikipedia/commons/thumb/1/1f/Gauss-sum.svg/800px-Gauss-sum.svg.png" width="500px" height="300px" />


> Ya derivamos la forma general del algoritmo de maximización de la esperanza, e hicimos un ejemplo sencillo. Es hora de ver como se aplica este algoritmo a ejemplos más interesantes.

> Comenzaremos por aplicarlo al problema de mezclas Gaussianas.

> **Objetivos:**
> - Aplicar el algoritmo de maximización de la esperanza al modelo de mezclas Gaussianas.

> **Referencias:**
> - Bayesian Methods for Machine Learning course, HSE University, Coursera.

## 1. Recordamos el problema

In [ ]:
# Importamos función para generar datos

# Importamos scipy.stats.multivariate_normal

# Importamos pyplot

# Importamos numpy


In [ ]:
# Generamos datos


In [ ]:
# Ajustamos parámetros

# Definimos VA


In [ ]:
# Datos
plt.scatter(data["income"], data["debt"], c="gray", alpha=0.6)

# Gaussiana 1
x = np.linspace(0, 6, 100)
y = np.linspace(0, 5, 100)
x, y = np.meshgrid(x, y)
z = X1.pdf(np.dstack([x, y]))
plt.contour(x, y, z)

# Gaussiana 2
x = np.linspace(0, 4, 100)
y = np.linspace(4, 9, 100)
x, y = np.meshgrid(x, y)
z = X2.pdf(np.dstack([x, y]))
plt.contour(x, y, z)

# Gaussiana 3
x = np.linspace(3, 8, 100)
y = np.linspace(2, 8, 100)
x, y = np.meshgrid(x, y)
z = X3.pdf(np.dstack([x, y]))
plt.contour(x, y, z)

plt.xlabel("Ingresos mensuales (x100k MXN)")
plt.ylabel("Deuda (x100k MXN)")

Recordamos que en un **modelo de mezclas Gaussianas** tenemos un conjunto de puntos el cual queremos modelar como una mezcla de Gaussianas:

$$
p(x | \theta) = \sum_{c=1}^{k} \pi_c \mathcal{N}(x | \mu_c, \Sigma_c).
$$

Los parámetros de este modelo son:

$$
\theta = \left\{(\pi_c, \mu_c, \Sigma_c): c=1,\dots, k \right\}.
$$

Desde una perspectiva probabilística, esto lo podemos modelar como un modelo de variable latente:

![latent](figures/latent_model.png)

con las siguientes distribuciones:

- $p(t_i=c | \theta) = \pi_c$

- $p(x_i | t_i=c, \theta) = \mathcal{N}(x_i | \mu_c, \Sigma_c)$

## 2. Conexión con algoritmo de maximización de la esperanza

Obtengamos las expresiones detalladas en los pasos del algoritmo de maximización de la esperanza.

### E-step

Para cada punto, debemos calcular la distribución posterior:

$$
q(t_i) = p(t_i|x_i, \theta).
$$

En este caso, usando el teorema de Bayes:

\begin{align}
p(t_i=c | x_i, \theta) & = \frac{p(x_i | t_i=c, \theta) p(t_i=c | \theta)}{p(x_i | \theta)} \\
                       & = \frac{\mathcal{N}(x_i | \mu_c, \Sigma_c) \pi_c}{\sum_{c=1}^{k} \pi_c \mathcal{N}(x | \mu_c, \Sigma_c)}. \\
\end{align}

### M-step

Debemos actualizar los parámetros de modo que maximicen la verosimilitud conjunta:

$$
\max_{\theta} \sum_{i=1}^{N} \mathbb{E}_{q(t_i)} \left[\log p(x_i, t_i| \theta)\right]
$$

Desarrollemos un poco la función a maximizar:

\begin{align}
\sum_{i=1}^{N} \mathbb{E}_{q(t_i)} \left[\log p(x_i, t_i| \theta)\right] 
& = \sum_{i=1}^{N} \sum_{c=1}^{k} q(t_i=c) \log \left[p(x_i| t_i=c \theta) p(t_i=c | \theta)\right] \\
& = \sum_{i=1}^{N} \sum_{c=1}^{k} q(t_i=c) \log \left[\mathcal{N}(x_i | \mu_c, \Sigma_c) \pi_c\right]
\end{align}

Para facilitar el entendimiento, resolvamos el caso escalar:

\begin{align}
\sum_{i=1}^{N} \mathbb{E}_{q(t_i)} \left[\log p(x_i, t_i| \theta)\right] 
& = \sum_{i=1}^{N} \sum_{c=1}^{k} q(t_i=c) \log \left[p(x_i| t_i=c, \theta) p(t_i=c | \theta)\right] \\
& = \sum_{i=1}^{N} \sum_{c=1}^{k} q(t_i=c) \log \left[\mathcal{N}(x_i | \mu_c, \sigma_c^2) \pi_c\right] \\
& = \sum_{i=1}^{N} \sum_{c=1}^{k} q(t_i=c) \log \left[\frac{\pi_c}{\sqrt{2 \pi \sigma_c^2}}\exp\left\{-\frac{(x_i - \mu_c)^2}{2\sigma_c^2}\right\}\right] \\
& = \sum_{i=1}^{N} \sum_{c=1}^{k} q(t_i=c) \left[\log \pi_c - \frac{1}{2}\log 2 \pi - \frac{1}{2}\log\sigma_c^2 -\frac{(x_i - \mu_c)^2}{2\sigma_c^2}\right] := f(\theta)\\
\end{align}

Derivamos respecto a $\mu_c$ e igualamos a cero:

\begin{align}
\frac{\partial f(\theta)}{\partial \mu_c} = \sum_{i=1}^{N} q(t_i=c)\frac{x_i - \mu_c}{\sigma_c^2} & = 0 \\
\Leftrightarrow \sum_{i=1}^{N} q(t_i=c)\left(x_i - \mu_c\right) & = 0 \\
\Leftrightarrow \mu_c & = \frac{\sum_{i=1}^{N} q(t_i=c)x_i}{\sum_{i=1}^{N} q(t_i=c)}.
\end{align}

Derivamos respecto a $\sigma_c^2$ e igualamos a cero:

\begin{align}
\frac{\partial f(\theta)}{\partial \sigma_c^2} = \sum_{i=1}^{N} q(t_i=c)\left[- \frac{1}{2\sigma_c^2}+\frac{(x_i - \mu_c)^2}{2(\sigma_c^2)^2}\right] & = 0 \\
\Leftrightarrow \sum_{i=1}^{N} q(t_i=c)\left[-1+\frac{(x_i - \mu_c)^2}{\sigma_c^2}\right] & = 0 \\
\Leftrightarrow \sigma_c^2 & = \frac{\sum_{i=1}^{N} q(t_i=c)(x_i - \mu_c)^2}{\sum_{i=1}^{N} q(t_i=c)}.
\end{align}

Para el caso de los parámetros $\pi_c$, tenemos que tener en cuenta la restricción $\sum_{c=1}^{k} \pi_c = 1$. El Lagrangiano es:

$$
L(\theta, \lambda) = f(\theta) - \lambda \left(\sum_{c=1}^{k} \pi_c - 1\right).
$$

Las derivadas respecto a $\pi_c$ y a $\lambda$, igualadas a cero, resulta en:

\begin{align}
\frac{\partial L(\theta, \lambda)}{\partial \pi_c} = \sum_{i=1}^{N} q(t_i=c) \frac{1}{\pi_c} - \lambda & = 0 \\
\Leftrightarrow \pi_c & = \frac{\sum_{i=1}^{N} q(t_i=c)}{\lambda}.
\end{align}

\begin{align}
\frac{\partial L(\theta, \lambda)}{\partial \lambda} = \sum_{c=1}^{k} \pi_c - 1 & = 0 \\
\Leftrightarrow \sum_{c=1}^{k} \frac{\sum_{i=1}^{N} q(t_i=c)}{\lambda} - 1 & = 0 \\
\Leftrightarrow \sum_{i=1}^{N} \underbrace{\sum_{c=1}^{k} q(t_i=c)}_{1} - \lambda & = 0 \\
\Leftrightarrow \lambda &= N.
\end{align}

Finalmente:

$$
\pi_c = \frac{\sum_{i=1}^{N} q(t_i=c)}{N}.
$$

**En resumen:**

- $\mu_c = \frac{\sum_{i=1}^{N} q(t_i=c)x_i}{\sum_{i=1}^{N} q(t_i=c)}$.
- $\sigma_c^2 = \frac{\sum_{i=1}^{N} q(t_i=c)(x_i - \mu_c)^2}{\sum_{i=1}^{N} q(t_i=c)}$.
- $\pi_c = \frac{\sum_{i=1}^{N} q(t_i=c)}{N}$.

___
#### Nota:

Acá hemos resuelto el caso escalar para facilitar el entendimiento, sin embargo, también es posible obtener soluciones cerradas en el caso multivariable.

En el caso multivariable ($x \in \mathbb{R}^n$), la función $f(\theta)$ es:

$$
f(\theta) = \sum_{i=1}^{N} \sum_{c=1}^{k} q(t_i=c) \left[\log \pi_c - \frac{1}{2}\log (2 \pi)^n + \frac{1}{2}\log \det(\Lambda_c) - \frac{1}{2}(x_i - \mu_c)^T \Lambda_c (x_i - \mu_c)\right],
$$

donde $\Lambda_c = \Sigma_c^{-1}$.

En este caso, es útil saber los siguientes gradientes:

1. Forma cuadrática respecto al vector ($y \in \mathbb{R}^n$, $Q = Q^T \in\mathbb{R}^{n \times n}$):
   $$
   \frac{\partial y^T Q y}{\partial y} = 2 Q y.
   $$
   
2. Forma cuadrática respecto a la matriz ($y \in \mathbb{R}^n$, $Q = Q^T \in\mathbb{R}^{n \times n}$):
   $$
   \frac{\partial y^T Q y}{\partial Q} = y y^T.
   $$
   
3. Gradiente del log-det ($Q = Q^T \in\mathbb{R}^{n \times n}$):
   $$
   \frac{\partial \log \det (Q)}{\partial Q} = Q^{-1}.
   $$

Con lo que se obtienen los siguientes resultados (análogos al caso escalar):

$$
\mu_c = \frac{\sum_{i=1}^{N} q(t_i=c)x_i}{\sum_{i=1}^{N} q(t_i=c)},
$$

$$
\Sigma_c = \frac{\sum_{i=1}^{N} q(t_i=c)(x_i - \mu_c) (x_i - \mu_c)^T}{\sum_{i=1}^{N} q(t_i=c)},
$$

$$
\pi_c = \frac{\sum_{i=1}^{N} q(t_i=c)}{N}.
$$
___

## 3. En resumen

**Entrenamiento de parámetros para GMM:**

1. Inicializar los parámetros:
   $$
   \theta^0 = \left\{(\pi_c^0, \mu_c^0, \Sigma_c^0): c=1,\dots, k \right\}.
   $$
   
   - $\pi_c^0$ se puede inicializar con una muestra de la distribución de Dirichlet.
   - $\mu_c^0$ se puede inicializar eligiendo aleatoriamente datos de la muestra para entrenar.
   - $\Sigma_c^0$ se puede inicializar con la matriz identidad, o como $R^T R$ donde $R$ es una matriz de números aleatorios.
   
2. Repetir hasta la convergencia ($j=0, 1, \dots$):

   - E-step: para todo $c=1,\dots,k$, para todo $i=1,\dots,N$,
     $$
     q^{j+1}(t_i = c) = \frac{\mathcal{N}(x_i | \mu_c^j, \Sigma_c^j) \pi_c^j}{\sum_{c=1}^{k} \pi_c^j \mathcal{N}(x | \mu_c^j, \Sigma_c^j)}
     $$
     
   - M-step: para todo $c=1,\dots,k$
     $$
     \mu_c^{j+1} = \frac{\sum_{i=1}^{N} q^{j+1}(t_i=c)x_i}{\sum_{i=1}^{N} q^{j+1}(t_i=c)},
     $$

     $$
     \Sigma_c^{j+1} = \frac{\sum_{i=1}^{N} q^{j+1}(t_i=c)(x_i - \mu_c) (x_i - \mu_c)^T}{\sum_{i=1}^{N} q^{j+1}(t_i=c)},
     $$

     $$
     \pi_c^{j+1} = \frac{\sum_{i=1}^{N} q^{j+1}(t_i=c)}{N}.
     $$
     

## 4. Sobre el ejemplo de clientes bancarios

In [ ]:
# Importamos sklearn.model_selection.train_test_split

# Importamos sklearn.mixture.GaussianMixture


In [ ]:
# GaussianMixture model help


In [ ]:
# Datos


In [ ]:
# Modelo de mezclas Gaussianas


# Entrenamiento


In [ ]:
# Parámetros óptimos


In [ ]:
# Gaussianas ajustadas


In [ ]:
# Datos
plt.scatter(data["income"], data["debt"], c="gray", alpha=0.6)

# Gaussiana 1
x = np.linspace(0, 10, 100)
y = np.linspace(0, 8, 100)
x, y = np.meshgrid(x, y)
z = X1.pdf(np.dstack([x, y]))
plt.contour(x, y, z)

# Gaussiana 2
x = np.linspace(0, 10, 100)
y = np.linspace(0, 8, 100)
x, y = np.meshgrid(x, y)
z = X2.pdf(np.dstack([x, y]))
plt.contour(x, y, z)

# Gaussiana 3
x = np.linspace(0, 10, 100)
y = np.linspace(0, 8, 100)
x, y = np.meshgrid(x, y)
z = X3.pdf(np.dstack([x, y]))
plt.contour(x, y, z)

plt.xlabel("Ingresos mensuales (x100k MXN)")
plt.ylabel("Deuda (x100k MXN)")

In [ ]:
# Por colores - respecto a la probabilidad de pertenencia


Una de las razones principales para considerar clustering probabilístico era la elección del número de clusters:

$$
p(x | \theta) = \sum_{c=1}^{k} \pi_c \mathcal{N}(x | \mu_c, \Sigma_c).
$$

In [ ]:
def log_likelihood_gmm(X, mu, sigma, pi):
    """
    Log-likelihood of the data wrt Gaussian Mixture Model.
    :param data: Data.
    :param mu: Means of the components of the GMM.
    :param sigma: Covariances of the components of the GMM.
    :param pi: Weights of the components of the GMM.
    :return: Log-likelihood of the data wrt GMM.
    """
    # Number of clusters
    k = mu.shape[0]
    # Number of points
    N = X.shape[0]
    
    # Individual likelihood of each point to each normal
    ind_likelihood = np.zeros((N, k))
    for j in range(k):
        ind_likelihood[:, j] = multivariate_normal.pdf(X, mean=mu[j, :], cov=sigma[j, :, :])
    
    # Log likelihood
    log_likelihood = np.log(ind_likelihood.dot(pi)).mean()
        
    return log_likelihood

In [ ]:
# División de datos

log_likelihood_train = []
log_likelihood_test = []
for k in range(2, 11):
    # Instanciamos el algoritmo
    
    
    # Entrenamos
    
    
    # Métrica con datos de entrenamiento
    
    
    # Métrica con datos de prueba
    

De esta manera, de acuerdo a la log-verosimilitud sobre el conjunto de pruebas, el número de componentes óptimo es:

In [ ]:
# Número de componentes óptimo


## ¡No hay overfitting!

<script>
  $(document).ready(function(){
    $('div.prompt').hide();
    $('div.back-to-top').hide();
    $('nav#menubar').hide();
    $('.breadcrumb').hide();
    $('.hidden-print').hide();
  });
</script>

<footer id="attribution" style="float:right; color:#808080; background:#fff;">
Created with Jupyter by Esteban Jiménez Rodríguez.
</footer>